# Migrate MongoDB database from `nmdc-schema` `v11.9.1` to `v11.10.0`

## Introduction

This notebook will be used to migrate the database from `nmdc-schema` `v11.9.1` ([released](https://github.com/microbiomedata/nmdc-schema/releases/tag/v11.9.1) July 16, 2025) to `v11.10.0` (released TBD).

### Notice

In each migration notebook between schema `v10.9.1` and `v11.3.0`, we dumped **all collections** from the Mongo database. We started doing that once migrations involved _collection_-level operations (i.e., creating, renaming, and deleting them), as opposed to only _document_-level operations.

In _this_ migration notebook, we dump only **specific collection** from the Mongo database. We opted to do this after understanding the scope of the `Migrator` class imported by this notebook. This eliminates some overhead from the migration process (i.e. dumping and restoring unrelated collections).

In [1]:
# Warning! The UCUM units migrator will dynamically traverse the schema looking for slots that have range QuantityValue.
# Confirm that the output of the following function is included in COLLECTION_NAMES:
#
# >>> from nmdc_schema.migrators.partials.migrator_from_11_9_1_to_11_10_0.migrator_from_11_9_1_to_11_10_0_part_1 import get_collection_names_with_qv_slots_from_schema
# >>> get_collection_names_with_qv_slots_from_schema()

COLLECTION_NAMES = list({
    'biosample_set',
    'material_processing_set',
    'configuration_set',
    'processed_sample_set',
    'storage_process_set'
})

## Prerequisites

### 1. Coordinate with stakeholders.

We will be enacting full Runtime and Database downtime for this migration. Ensure stakeholders are aware of that.

### 2. Set up notebook environment.

Here, you'll prepare an environment for running this notebook.

1. Start a **MongoDB server** on your local machine (and ensure it does **not** already contain a database having the name specified in the notebook configuration file).
    1. You can start a [Docker](https://hub.docker.com/_/mongo)-based MongoDB server at `localhost:27055` by running the following command. A MongoDB server started this way will be accessible without a username or password.


In [23]:
%%bash
# The 11.10.0 migrator uses transactions, so start the MongoDB server with a replica set.
docker run \
    --rm \
    --detach \
    --name mongo-migration-transformer \
    -p 27055:27017 \
    -v ./.docker/01-init.js:/docker-entrypoint-initdb.d/01-init.js \
    mongo:8.0.4 \
    --replSet rs0

2187509d84b77d25ef46dd14d66b1aa59da75c3644d0e9bc9920eb5aacaac564


2. Create and populate a **notebook configuration file** named `.notebook.env`.
   > You can use `.notebook.env.example` as a template.

## Procedure

### Install Python packages

In this step, you'll [install](https://saturncloud.io/blog/what-is-the-difference-between-and-in-jupyter-notebooks/) the Python packages upon which this notebook depends.

> Note: If the output of this cell says "Note: you may need to restart the kernel to use updated packages", restart the kernel (not the notebook cells), then proceed to the next cell.

##### References

| Description                                                                     | Link                                                   |
|---------------------------------------------------------------------------------|--------------------------------------------------------|
| NMDC Schema PyPI package | https://pypi.org/project/nmdc-schema                   |
| How to `pip install` from a Git branch<br>instead of PyPI                       | https://stackoverflow.com/a/20101940                   |

In [38]:
%pip install --upgrade pip
%pip install -r requirements.txt
%pip install nmdc-schema==11.10.0rc2

Note: you may need to restart the kernel to use updated packages.
  Using cached linkml-1.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached linkml_runtime-1.8.2-py3-none-any.whl.metadata (3.6 kB)
Using cached linkml-1.8.5-py3-none-any.whl (313 kB)
Using cached linkml_runtime-1.8.2-py3-none-any.whl (577 kB)
  Attempting uninstall: linkml-runtime
    Found existing installation: linkml-runtime 1.9.4
    Uninstalling linkml-runtime-1.9.4:
      Successfully uninstalled linkml-runtime-1.9.4
  Attempting uninstall: linkml━━━━━━━━━━━━━━━━━━ 0/2 [linkml-runtime]
    Found existing installation: linkml 1.9.3 0/2 [linkml-runtime]
    Uninstalling linkml-1.9.3:━━━━━━━━━━━━━━ 0/2 [linkml-runtime]
      Successfully uninstalled linkml-1.9.3━ 0/2 [linkml-runtime]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [linkml]2m1/2 [linkml]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency 

### Import Python dependencies

Import the Python objects upon which this notebook depends.

#### References

| Description                            | Link                                                                                                  |
|----------------------------------------|-------------------------------------------------------------------------------------------------------|
| Dynamically importing a Python module  | [`importlib.import_module`](https://docs.python.org/3/library/importlib.html#importlib.import_module) |
| Confirming something is a Python class | [`inspect.isclass`](https://docs.python.org/3/library/inspect.html#inspect.isclass)                   |

In [3]:
MIGRATOR_MODULE_NAME = "migrator_from_11_9_1_to_11_10_0"

In [4]:
# Standard library packages:
import subprocess
from typing import List
import importlib
from inspect import isclass

# Third-party packages:
import pymongo
from linkml.validator import Validator, ValidationReport
from linkml.validator.plugins import JsonschemaValidationPlugin
from nmdc_schema.nmdc_data import get_nmdc_schema_definition
from nmdc_schema.migrators.adapters.mongo_adapter import MongoAdapter
from linkml_runtime import SchemaView

# First-party packages:
from helpers import Config, setup_logger, get_collection_names_from_schema, derive_schema_class_name_from_document
from bookkeeper import Bookkeeper, MigrationEvent

# Dynamic imports:
migrator_module = importlib.import_module(f".{MIGRATOR_MODULE_NAME}", package="nmdc_schema.migrators")
Migrator = getattr(migrator_module, "Migrator")  # gets the class
assert isclass(Migrator), "Failed to import Migrator class."

### Parse configuration files

Parse the notebook and Mongo configuration files.

In [5]:
cfg = Config()

# Define some aliases we can use to make the shell commands in this notebook easier to read.
mongodump = cfg.mongodump_path
mongorestore = cfg.mongorestore_path
mongosh = cfg.mongosh_path

# Make the base CLI options for Mongo shell commands.
origin_mongo_cli_base_options = Config.make_mongo_cli_base_options(
    mongo_host=cfg.origin_mongo_host,
    mongo_port=cfg.origin_mongo_port,
    mongo_username=cfg.origin_mongo_username,
    mongo_password=cfg.origin_mongo_password,
)
transformer_mongo_cli_base_options = Config.make_mongo_cli_base_options(
    mongo_host=cfg.transformer_mongo_host,
    mongo_port=cfg.transformer_mongo_port,
    mongo_username=cfg.transformer_mongo_username,
    mongo_password=cfg.transformer_mongo_password,
)

# Perform a sanity test of the application paths.
!{mongodump} --version
!{mongorestore} --version
!{mongosh} --version

mongodump version: 100.7.2
git version: db8c5c49905eab242db186a32e84afd9e5aa9837
Go version: go1.19.9
   os: darwin
   arch: arm64
   compiler: gc
mongorestore version: 100.7.2
git version: db8c5c49905eab242db186a32e84afd9e5aa9837
Go version: go1.19.9
   os: darwin
   arch: arm64
   compiler: gc
2.3.9


### Create MongoDB clients

Create MongoDB clients you can use to access the "origin" and "transformer" MongoDB servers.

In [6]:
# Mongo client for "origin" MongoDB server.
origin_mongo_client = pymongo.MongoClient(host=cfg.origin_mongo_host,
                                          port=int(cfg.origin_mongo_port),
                                          username=cfg.origin_mongo_username,
                                          password=cfg.origin_mongo_password,
                                          directConnection=True)

# Mongo client for "transformer" MongoDB server.
transformer_mongo_client = pymongo.MongoClient(host=cfg.transformer_mongo_host,
                                               port=int(cfg.transformer_mongo_port),
                                               username=cfg.transformer_mongo_username,
                                               password=cfg.transformer_mongo_password,
                                               directConnection=True)

# Perform sanity tests of those MongoDB clients' abilities to access their respective MongoDB servers.
with pymongo.timeout(3):
    # Display the MongoDB server version (running on the "origin" Mongo server).
    print("Origin Mongo server version:      " + origin_mongo_client.server_info()["version"])

    # Sanity test: Ensure the origin database exists.
    assert cfg.origin_mongo_database_name in origin_mongo_client.list_database_names(), "Origin database does not exist."

    # Display the MongoDB server version (running on the "transformer" Mongo server).
    print("Transformer Mongo server version: " + transformer_mongo_client.server_info()["version"])

    # Sanity test: Ensure the transformation database does not exist.
    assert cfg.transformer_mongo_database_name not in transformer_mongo_client.list_database_names(), "Transformation database already exists."

{"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "buildinfo", "topologyDescription": "<TopologyDescription id: 6896678ecd5a871af82e2e63, topology_type: Single, servers: [<ServerDescription ('localhost', 59639) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "6896678ecd5a871af82e2e63"}, "remainingTimeMS": 2}


Origin Mongo server version:      8.0.4
Transformer Mongo server version: 8.0.4


Delete the transformer database from the transformer MongoDB server if that database already exists there (e.g. if it was left over from an experiment).

#### References

| Description                  | Link                                                          |
|------------------------------|---------------------------------------------------------------|
| Python's `subprocess` module | https://docs.python.org/3/library/subprocess.html             |
| `mongosh` CLI options        | https://www.mongodb.com/docs/mongodb-shell/reference/options/ |

In [7]:
# Note: I run this command via Python's `subprocess` module instead of via an IPython magic `!` command
#       because I expect to eventually use regular Python scripts—not Python notebooks—for migrations.
shell_command = f"""
  {cfg.mongosh_path} {transformer_mongo_cli_base_options} \
      --eval 'use {cfg.transformer_mongo_database_name}' \
      --eval 'db.dropDatabase()' \
      --quiet
"""
completed_process = subprocess.run(shell_command, shell=True)
print(f"\nReturn code: {completed_process.returncode}")

{ ok: 1, dropped: 'nmdc' }

Return code: 0


### Create validator

In this step, you'll create a validator that can be used to check whether data conforms to the NMDC Schema. You'll use it later, to do that.

#### References

| Description                  | Link                                                                         |
|------------------------------|------------------------------------------------------------------------------|
| LinkML's `Validator` class   | https://linkml.io/linkml/code/validator.html#linkml.validator.Validator      |
| Validating data using LinkML | https://linkml.io/linkml/data/validating-data.html#validation-in-python-code |

In [8]:
schema_definition = get_nmdc_schema_definition()
validator = Validator(
    schema=schema_definition,
    validation_plugins=[JsonschemaValidationPlugin(closed=True)],
)

# Perform a sanity test of the validator.
assert callable(validator.validate), "Failed to instantiate a validator"

### Create SchemaView

In this step, you'll instantiate a `SchemaView` that is bound to the destination schema.

#### References

| Description                 | Link                                                |
|-----------------------------|-----------------------------------------------------|
| LinkML's `SchemaView` class | https://linkml.io/linkml/developers/schemaview.html |

In [9]:
schema_view = SchemaView(get_nmdc_schema_definition())

# As a sanity test, confirm we can use the `SchemaView` instance to access a schema class.
schema_view.get_class(class_name="Database")["name"]

'Database'

### Revoke access from the "origin" MongoDB server

We revoke both "write" and "read" access to the server.

#### Rationale

We revoke "write" access so people don't make changes to the original data while the migration is happening, given that the migration ends with an overwriting of the original data (which would wipe out any changes made in the meantime).

We also revoke "read" access. The revocation of "read" access is technically optional, but (a) the JavaScript mongosh script will be easier for me to maintain if it revokes everything and (b) this prevents people from reading data during the restore step, during which the database may not be self-consistent.

#### References

| Description                    | Link                                                      |
|--------------------------------|-----------------------------------------------------------|
| Running a script via `mongosh` | https://www.mongodb.com/docs/mongodb-shell/write-scripts/ |

In [10]:
shell_command = f"""
  {cfg.mongosh_path} {origin_mongo_cli_base_options} \
      --file='mongosh-scripts/revoke-privileges.mongo.js' \
      --quiet
"""
completed_process = subprocess.run(shell_command, shell=True)
print(f"\nReturn code: {completed_process.returncode}")


Return code: 1


MongoServerError: Could not find role: nmdc_reader@admin


### Delete obsolete dumps from previous migrations

Delete any existing dumps before we create new ones in this notebook. This is so the dumps you generate with this notebook do not get merged with any unrelated ones.

In [11]:
!rm -rf {cfg.origin_dump_folder_path}
!rm -rf {cfg.transformer_dump_folder_path}

### Dump collection(s) from the "origin" MongoDB server

Use `mongodump` to dump specific collection(s) **from** the "origin" MongoDB server **into** a local directory.

> Note: The reason we do this in a loop is that the `mongodump` program does not allow the user to specify multiple collections to dump. A distinct collection will be dumped on each iteration of the loop.


In [12]:
# Dump the specified collections from the "origin" database.
for collection_name in COLLECTION_NAMES:
  shell_command = f"""
    {mongodump} {origin_mongo_cli_base_options} \
        --db='{cfg.origin_mongo_database_name}' \
        --out='{cfg.origin_dump_folder_path}' \
        --gzip \
        --collection='{collection_name}'
  """
  completed_process = subprocess.run(shell_command, shell=True)
  print(f"\nReturn code: {completed_process.returncode}")

2025-08-08T14:09:54.140-0700	writing nmdc.storage_process_set to mongodump.origin.out/nmdc/storage_process_set.bson.gz
2025-08-08T14:09:54.142-0700	done dumping nmdc.storage_process_set (0 documents)
2025-08-08T14:09:54.202-0700	writing nmdc.biosample_set to mongodump.origin.out/nmdc/biosample_set.bson.gz



Return code: 0


2025-08-08T14:09:54.643-0700	done dumping nmdc.biosample_set (13248 documents)
2025-08-08T14:09:54.704-0700	writing nmdc.configuration_set to mongodump.origin.out/nmdc/configuration_set.bson.gz
2025-08-08T14:09:54.706-0700	done dumping nmdc.configuration_set (19 documents)
2025-08-08T14:09:54.744-0700	writing nmdc.processed_sample_set to mongodump.origin.out/nmdc/processed_sample_set.bson.gz
2025-08-08T14:09:54.794-0700	done dumping nmdc.processed_sample_set (10043 documents)
2025-08-08T14:09:54.844-0700	writing nmdc.material_processing_set to mongodump.origin.out/nmdc/material_processing_set.bson.gz



Return code: 0

Return code: 0

Return code: 0

Return code: 0


2025-08-08T14:09:54.940-0700	done dumping nmdc.material_processing_set (9242 documents)


### Load the dumped collection(s) into the "transformer" MongoDB server

Use `mongorestore` to load the dumped collection(s) **from** the local directory **into** the "transformer" MongoDB server.

References:
- https://www.mongodb.com/docs/database-tools/mongorestore/#std-option-mongorestore
- https://www.mongodb.com/docs/database-tools/mongorestore/mongorestore-examples/#copy-clone-a-database

In [13]:
# Restore the dumped collections to the "transformer" MongoDB server.
shell_command = f"""
  {mongorestore} {transformer_mongo_cli_base_options} \
      --nsFrom='{cfg.origin_mongo_database_name}.*' \
      --nsTo='{cfg.transformer_mongo_database_name}.*' \
      --dir='{cfg.origin_dump_folder_path}' \
      --stopOnError \
      --drop \
      --gzip
"""
completed_process = subprocess.run(shell_command, shell=True)
print(f"\nReturn code: {completed_process.returncode}")

2025-08-08T14:10:03.259-0700	preparing collections to restore from
2025-08-08T14:10:03.260-0700	reading metadata for nmdc.configuration_set from mongodump.origin.out/nmdc/configuration_set.metadata.json.gz
2025-08-08T14:10:03.260-0700	reading metadata for nmdc.material_processing_set from mongodump.origin.out/nmdc/material_processing_set.metadata.json.gz
2025-08-08T14:10:03.260-0700	reading metadata for nmdc.processed_sample_set from mongodump.origin.out/nmdc/processed_sample_set.metadata.json.gz
2025-08-08T14:10:03.260-0700	reading metadata for nmdc.storage_process_set from mongodump.origin.out/nmdc/storage_process_set.metadata.json.gz
2025-08-08T14:10:03.260-0700	reading metadata for nmdc.biosample_set from mongodump.origin.out/nmdc/biosample_set.metadata.json.gz
2025-08-08T14:10:03.275-0700	restoring nmdc.biosample_set from mongodump.origin.out/nmdc/biosample_set.bson.gz
2025-08-08T14:10:03.280-0700	restoring nmdc.material_processing_set from mongodump.origin.out/nmdc/material_proce


Return code: 0


2025-08-08T14:10:03.903-0700	32552 document(s) restored successfully. 0 document(s) failed to restore.


### Transform the collections within the "transformer" MongoDB server

Use the migrator to transform the collections in the "transformer" database.

> Reminder: The database transformation functions are defined in the `nmdc-schema` Python package installed earlier.

> Reminder: The "origin" database is **not** affected by this step.

In [14]:
# Instantiate a MongoAdapter bound to the "transformer" database.
adapter = MongoAdapter(
    database=transformer_mongo_client[cfg.transformer_mongo_database_name],
    on_collection_created=lambda name: print(f'Created collection "{name}"'),
    on_collection_renamed=lambda old_name, name: print(f'Renamed collection "{old_name}" to "{name}"'),
    on_collection_deleted=lambda name: print(f'Deleted collection "{name}"'),
)

# Instantiate a Migrator bound to that adapter.
logger = setup_logger()
migrator = Migrator(adapter=adapter, logger=logger)

# Execute the Migrator's `upgrade` method to perform the migration.
migrator.upgrade(commit_changes=True)

Running migrator 1 of 1
Migrating from 11.9.1 to 11.10.0
MIGRATION SUMMARY
RECORDS UPDATED
----------------------------------------
Class                          SubClassType              Slot                      Source Value    Conformant   Not          Updated  Target Value   
                                                                                                                Conformant                           
-----------------------------------------------------------------------------------------------------------------------------------------------------------
nmdc:Biosample                                           abs_air_humidity          kPa             192          0            0        kPa            
nmdc:Biosample                                           ammonium                  micromol/L      0            275          275      umol/L         
nmdc:Biosample                                           ammonium                  mg/L            45           

### Validate the transformed documents

Now that we have transformed the database, validate each document in each collection in the "transformer" MongoDB server.

In [15]:
# Get the names of all collections.
collection_names: List[str] = get_collection_names_from_schema(schema_view)

# Ensure that, if the (large) "functional_annotation_agg" collection is present in `collection_names`,
# it goes at the end of the list we process. That way, we can find out about validation errors in
# other collections without having to wait for that (large) collection to be validated.
ordered_collection_names = sorted(collection_names.copy())
large_collection_name = "functional_annotation_agg"
if large_collection_name in ordered_collection_names:
    ordered_collection_names = list(filter(lambda n: n != large_collection_name, ordered_collection_names))
    ordered_collection_names.append(large_collection_name)  # puts it last

# Process each collection.
for collection_name in ordered_collection_names:
    collection = transformer_mongo_client[cfg.transformer_mongo_database_name][collection_name]
    num_documents_in_collection = collection.count_documents({})
    print(f"Validating collection {collection_name} ({num_documents_in_collection} documents) [", end="")  # no newline

    # Calculate how often we'll display a tick mark (i.e. a sign of life).
    num_documents_per_tick = num_documents_in_collection * 0.10  # one tenth of the total
    num_documents_since_last_tick = 0

    for document in collection.find():
        # Validate the transformed document.
        #
        # Reference: https://github.com/microbiomedata/nmdc-schema/blob/main/src/docs/schema-validation.md
        #
        # Note: Dictionaries originating as Mongo documents include a Mongo-generated key named `_id`. However,
        #       the NMDC Schema does not describe that key and, indeed, data validators consider dictionaries
        #       containing that key to be invalid with respect to the NMDC Schema. So, here, we validate a
        #       copy (i.e. a shallow copy) of the document that lacks that specific key.
        #
        # Note: The reason we don't use a progress bar library such as `rich[jupyter]`, `tqdm`, or `ipywidgets`
        #       is that _PyCharm's_ Jupyter Notebook integration doesn't fully work with any of them. :(
        #
        schema_class_name = derive_schema_class_name_from_document(schema_view=schema_view, document=document)
        document_without_underscore_id_key = {key: value for key, value in document.items() if key != "_id"}
        validation_report: ValidationReport = validator.validate(document_without_underscore_id_key, schema_class_name)
        if len(validation_report.results) > 0:
            result_messages = [result.message for result in validation_report.results]
            raise TypeError(f"Document is invalid.\n{result_messages=}\n{document_without_underscore_id_key=}")

        # Display a tick mark if we have validated enough documents since we last displayed one.
        num_documents_since_last_tick += 1
        if num_documents_since_last_tick >= num_documents_per_tick:
            num_documents_since_last_tick = 0
            print(".", end="")  # no newline

    print("]")

Validating collection biosample_set (13248 documents) [.........]
Validating collection calibration_set (0 documents) []
Validating collection collecting_biosamples_from_site_set (0 documents) []
Validating collection configuration_set (19 documents) [.........]
Validating collection data_generation_set (0 documents) []
Validating collection data_object_set (0 documents) []
Validating collection field_research_site_set (0 documents) []
Validating collection functional_annotation_set (0 documents) []
Validating collection genome_feature_set (0 documents) []
Validating collection instrument_set (0 documents) []
Validating collection manifest_set (0 documents) []
Validating collection material_processing_set (9242 documents) [.........]
Validating collection processed_sample_set (10043 documents) [.........]
Validating collection storage_process_set (0 documents) []
Validating collection study_set (0 documents) []
Validating collection workflow_execution_set (0 documents) []
Validating co

### Dump the collections from the "transformer" MongoDB server

Now that the collections have been transformed and validated, dump them **from** the "transformer" MongoDB server **into** a local directory.

In [16]:
# Dump the database from the "transformer" MongoDB server.
shell_command = f"""
  {mongodump} {transformer_mongo_cli_base_options} \
      --db='{cfg.transformer_mongo_database_name}' \
      --out='{cfg.transformer_dump_folder_path}' \
      --gzip
"""
completed_process = subprocess.run(shell_command, shell=True)
print(f"\nReturn code: {completed_process.returncode}")

2025-08-08T14:12:12.081-0700	writing nmdc.biosample_set to mongodump.transformer.out/nmdc/biosample_set.bson.gz
2025-08-08T14:12:12.082-0700	writing nmdc.processed_sample_set to mongodump.transformer.out/nmdc/processed_sample_set.bson.gz
2025-08-08T14:12:12.082-0700	writing nmdc.material_processing_set to mongodump.transformer.out/nmdc/material_processing_set.bson.gz
2025-08-08T14:12:12.087-0700	writing nmdc.configuration_set to mongodump.transformer.out/nmdc/configuration_set.bson.gz
2025-08-08T14:12:12.089-0700	done dumping nmdc.configuration_set (19 documents)
2025-08-08T14:12:12.090-0700	writing nmdc.storage_process_set to mongodump.transformer.out/nmdc/storage_process_set.bson.gz
2025-08-08T14:12:12.094-0700	done dumping nmdc.storage_process_set (0 documents)
2025-08-08T14:12:12.127-0700	done dumping nmdc.processed_sample_set (10043 documents)
2025-08-08T14:12:12.155-0700	done dumping nmdc.material_processing_set (9242 documents)



Return code: 0


2025-08-08T14:12:12.306-0700	done dumping nmdc.biosample_set (13248 documents)


### Create a bookkeeper

Create a `Bookkeeper` that can be used to document migration events in the "origin" server.

In [17]:
bookkeeper = Bookkeeper(mongo_client=origin_mongo_client)

### Indicate — on the "origin" server — that the migration is underway

Add an entry to the migration log collection to indicate that this migration has started.

In [18]:
bookkeeper.record_migration_event(migrator=migrator, event=MigrationEvent.MIGRATION_STARTED)

### Skipped: Drop the original collections from the "origin" MongoDB server

Note: This step is necessary for migrations where collections are being renamed or deleted. (The `--drop` option of `mongorestore` would only drop collections that exist in the dump being restored, which would not include renamed or deleted collections.)

In the case of _this_ migration, no collections are being renamed or deleted. So, we can skip this step. The collections that the migrator _did_ transform, will still be dropped when we run `mongorestore` with the `--drop` option later in this notebook.


In [19]:
print("skipped")

# shell_command = f"""
#   {cfg.mongosh_path} {origin_mongo_cli_base_options} \
#       --eval 'use {cfg.origin_mongo_database_name}' \
#       --eval 'db.dropDatabase()'
# """
# completed_process = subprocess.run(shell_command, shell=True)
# print(f"\nReturn code: {completed_process.returncode}")

skipped


### Load the collections into the "origin" MongoDB server

Load the transformed collections into the "origin" MongoDB server.

In [20]:
# Load the transformed collections into the origin server, replacing any same-named ones that are there.
shell_command = f"""
  {mongorestore} {origin_mongo_cli_base_options} \
      --nsFrom='{cfg.transformer_mongo_database_name}.*' \
      --nsTo='{cfg.origin_mongo_database_name}.*' \
      --dir='{cfg.transformer_dump_folder_path}' \
      --stopOnError \
      --verbose \
      --drop \
      --gzip
"""
completed_process = subprocess.run(shell_command, shell=True)
print(f"\nReturn code: {completed_process.returncode}")

2025-08-08T14:12:25.128-0700	using --dir flag instead of arguments
2025-08-08T14:12:25.128-0700	using write concern: &{majority false 0}
2025-08-08T14:12:25.146-0700	preparing collections to restore from
2025-08-08T14:12:25.147-0700	found collection nmdc.biosample_set bson to restore to nmdc.biosample_set
2025-08-08T14:12:25.147-0700	found collection metadata from nmdc.biosample_set to restore to nmdc.biosample_set
2025-08-08T14:12:25.147-0700	found collection nmdc.configuration_set bson to restore to nmdc.configuration_set
2025-08-08T14:12:25.147-0700	found collection metadata from nmdc.configuration_set to restore to nmdc.configuration_set
2025-08-08T14:12:25.147-0700	found collection nmdc.material_processing_set bson to restore to nmdc.material_processing_set
2025-08-08T14:12:25.147-0700	found collection metadata from nmdc.material_processing_set to restore to nmdc.material_processing_set
2025-08-08T14:12:25.147-0700	found collection nmdc.processed_sample_set bson to restore to nmdc


Return code: 0


2025-08-08T14:12:26.344-0700	32552 document(s) restored successfully. 0 document(s) failed to restore.


### Indicate that the migration is complete

Add an entry to the migration log collection to indicate that this migration is complete.

In [21]:
bookkeeper.record_migration_event(migrator=migrator, event=MigrationEvent.MIGRATION_COMPLETED)

### Restore access to the "origin" MongoDB server

This effectively un-does the access revocation that we did earlier.

In [22]:
shell_command = f"""
  {cfg.mongosh_path} {origin_mongo_cli_base_options} \
      --file='mongosh-scripts/restore-privileges.mongo.js' \
      --quiet
"""
completed_process = subprocess.run(shell_command, shell=True)
print(f"\nReturn code: {completed_process.returncode}")


Return code: 1


MongoServerError: Could not find role: nmdc_runtime@admin
